In [1]:
from keras.datasets import cifar10
from matplotlib import pyplot
import cv2 as cv
import numpy as np
from sklearn.cluster import KMeans
from scipy.cluster.vq import *
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import sklearn.metrics as metrics

In [2]:
(trainX, trainy), (testX, testy) = cifar10.load_data()

A local file was found, but it seems to be incomplete or outdated because the auto file hash does not match the original value of 6d958be074577803d12ecdefd02955f39262c83c16fe9348329d7fe0b5c001ce so we will re-download the data.
  3399680/170498071 [..............................] - ETA: 13:45

In [ ]:
def getFeatures(images,labels,method=None):

    if method == 'sift':
        descriptor = cv.SIFT_create()
    elif method == 'surf':
        descriptor = cv.SURF_create()
    desc_lst=[]
    labels_lst = []
    cnt = 0
    kpts_lst=[]

    for img in images:
      img = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
      #img = cv.resize(img,(150,150),interpolation=cv.INTER_AREA)
      kpts, desc = descriptor.detectAndCompute(img,None)
      if desc is not None:
        desc_lst.append(desc)
        kpts_lst.append(kpts)
        labels_lst.append(int(labels[cnt]))
      cnt = cnt + 1
    return kpts_lst,desc_lst,labels_lst


In [ ]:
kpts_lst,desc_lst,labels_lst = getFeatures(np.concatenate((trainX,testX),axis=0),np.concatenate((trainy,testy)),'sift')

In [ ]:
def getDescriptorStack(desc_lst):
  descriptors_stacked = desc_lst[0]
  for desc in desc_lst[1:]:
    descriptors_stacked = np.vstack((descriptors_stacked,desc))
  return descriptors_stacked

In [ ]:
descriptors_lst = getDescriptorStack(desc_lst)

In [ ]:
k = 80
voc,variance = kmeans(descriptors_lst,k,1)

In [ ]:
imgfeatures = np.zeros((len(labels_lst),k),"float32")
for i in range(len(labels_lst)):
  words,distance = vq(desc_lst[i],voc)
  for j in words:
    imgfeatures[i][j] +=1

In [ ]:
imgfeatures = StandardScaler().fit_transform(imgfeatures)

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(imgfeatures,labels_lst,test_size=0.1,random_state = 11)

In [ ]:

from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(xtrain, ytrain)
ypred = clf.predict(xtest)
metrics.accuracy_score(ytest, ypred)

0.2573480293921176